In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import Image
import cv2
import albumentations as A  # our data augmentation library

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm # progress bar
from torchvision.utils import draw_bounding_boxes

import tkinter as tk
from tkinter import ttk, Frame, Button, Canvas, Label
from tkinter.filedialog import askopenfilename
import sys
import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import ttk, Frame, Button, Canvas, Label
from tkinter.filedialog import askopenfilename
from PIL import ImageTk, Image
import sys

from torchvision.io import read_image
import tensorflow as tf

!pip install pycocotools
from pycocotools.coco import COCO
from albumentations.pytorch import ToTensorV2

C:\Users\krusl\anaconda3\envs\project2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
classes = ['rov', 'plant', 'animal_fish', 'animal_starfish', 'animal_shells', 'animal_crab', 'animal_eel', 'animal_etc', 'trash_etc', 'trash_fabric', 'trash_fishing_gear', 'trash_metal', 'trash_paper', 'trash_plastic', 'trash_rubber', 'trash_wood']


model = torch.load("C:/Users/krusl/Downloads/FasterRCNN2/model.pt")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

path = ""


def resizer(url):
    imagg = Image.open(url).convert('RGB')
    resized_imagg = imagg.resize((1230, 820), Image.ANTIALIAS)
    imagg = ImageTk.PhotoImage(resized_imagg)
    return imagg

def b1():
    panel.delete("all")
    filename = askopenfilename()
    global path
    path = filename
    img = resizer(filename)
    panel.create_image(615, 410, anchor="c", image=img)
    panel.image = img
    
def learning():
    if combobox.get() == "FasterRCNN":  
        os.chdir("C:/Users/krusl/Downloads/FasterRCNN2/")
        pic = read_image(path)
        img_int = torch.tensor(pic, dtype=torch.uint8)
        with torch.no_grad():
            prediction = model([pic.div(255).to(device)])
            pred = prediction[0]
        fig = plt.figure(figsize=(14, 10))
        plt.imshow(draw_bounding_boxes(img_int,
            pred['boxes'][pred['scores'] > 0.5],
            [classes[i] for i in pred['labels'][pred['scores'] > 0.5].tolist()], width=4).permute(1, 2, 0))
        plt.xticks([])
        plt.yticks([])
        os.chdir("C:/Users/krusl/Downloads/FasterRCNN2/output/")
        num_folders = len(os.listdir("C:/Users/krusl/Downloads/FasterRCNN2/output/"))
        tmp = f"{num_folders}"
        plt.savefig('saved_figure' + tmp + ".png", bbox_inches="tight")
        img = resizer("C:/Users/krusl/Downloads/FasterRCNN2/output/saved_figure" + tmp + ".png")
        panel.create_image(615, 410, anchor="c", image=img)
        panel.image = img
    elif combobox.get() == "Yolo5":
        os.chdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/")
        !python detect.py --source {path}  --weights "v1.pt" --conf 0.5 
        num_folders = len(os.listdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/"))
        tmp = "exp" + f"{num_folders}"
        tmp2 = path[path.rfind('/'):]
        img = resizer("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/" + tmp + tmp2)
        panel.create_image(615, 410, anchor="c", image=img)
        panel.image = img
    elif combobox.get() == "Yolo5v2":
        os.chdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/")
        !python detect.py --source {path}  --weights "best.pt" --conf 0.5
        num_folders = len(os.listdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/"))
        tmp = "exp" + f"{num_folders}"
        tmp2 = path[path.rfind('/'):]
        img = resizer("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/" + tmp + tmp2)
        panel.create_image(615, 410, anchor="c", image=img)
        panel.image = img
    elif combobox.get() == "Yolo5v3":
        os.chdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/")
        !python detect.py --source {path}  --weights "v3.pt" --conf 0.5
        num_folders = len(os.listdir("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/"))
        tmp = "exp" + f"{num_folders}"
        tmp2 = path[path.rfind('/'):]
        img = resizer("C:/Users/krusl/Downloads/yolo5/yolov5-master/runs/detect/" + tmp + tmp2)
        panel.create_image(615, 410, anchor="c", image=img)
        panel.image = img
    

#объявление объектов
models = ["FasterRCNN", "Yolo5", "Yolo5v2", "Yolo5v3"]
window = tk.Tk()
window.title("Добро пожаловать")
window.minsize(1400, 900)
window.config(bg="skyblue")

left_frame = Frame(window, width=150, height= 820)
right_frame = Frame(window, width=1250, height=820)
right_frame.grid(row=0, column=1, padx=10, pady=5)
left_frame.grid(row=0, column=0, padx=10, pady=5)


btn1 = Button(left_frame, text='Выбрать картинку', command=b1)
btn1.grid(row=2, column=0, padx=20, pady=20)
btn2 = Button(left_frame, text= 'Запустить', command=learning)
btn2.grid(row=4, column=0, padx=20, pady=20)
l = Label(left_frame, text = "Выбор модели")
l.grid(row=0, column=0, padx=20, pady=20)
combobox = ttk.Combobox(left_frame, values=models, state="readonly")
combobox.grid(row=1, column=0, padx=20, pady=20)

panel = Canvas(right_frame, width = 1230, height=820)
panel.pack(fill="both", expand=True)
img = resizer("C:/Users/krusl/Downloads/FasterRCNN2/gray.jpg")
panel.create_image(615, 410, anchor="c", image=img)
panel.grid()

window.mainloop()





detect: weights=['v1.pt'], source=C:/Users/krusl/Downloads/FasterRCNN2/test/test.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2022-11-23 Python-3.9.13 torch-1.14.0.dev20221113 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86274349 parameters, 0 gradients
image 1/1 C:\Users\krusl\Downloads\FasterRCNN2\test\test.jpg: 480x640 1 animal_eel, 2 trash_etcs, 108.1ms
Speed: 8.0ms pre-process, 108.1ms inference, 109.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp40
detect: weights=['best.pt'], source=C:/Users/krusl/Downloads/FasterRCNN2/test/test.jpg, data=data\coco128